This needs to run in SQL Azure Studio(SAS), not VS code as we need to change the notebook kernel type to SQL in SAS to run it (can't change the kernel in VScode??).
This is a version of https://github.com/microsoft/sqlworkshops-sql2019workshop/blob/master/sql2019workshop/04_Availability/adr/basic_adr.ipynb

Step 1: create a test db and a table with 100,000 rows. Took 24 secs.

In [25]:
USE master
GO

IF  EXISTS (SELECT name FROM sys.databases WHERE name = N'gocowboys')
alter DATABASE gocowboys SET SINGLE_USER WITH ROLLBACK IMMEDIATE

DROP DATABASE IF EXISTS gocowboys
GO

CREATE DATABASE gocowboys
ON PRIMARY
(NAME = N'gocowboys_primary', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.SQL2019\MSSQL\DATA\gocowboys.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
-- (NAME = N'gocowboys_primary', FILENAME = '/var/opt/mssql/DATA/gocowboys.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'gocowboys_Log', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.SQL2019\MSSQL\DATA\gocowboys_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
-- (NAME = N'gocowboys_Log', FILENAME = '/var/opt/mssql/DATA/gocowboys_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE gocowboys SET RECOVERY SIMPLE
GO
USE gocowboys
GO
DROP TABLE IF EXISTS howboutthemcowboys
GO
CREATE TABLE howboutthemcowboys 
(
    playerid int primary key clustered, 
    playername char(7000) not null
);

GO
SET NOCOUNT ON;
GO

BEGIN TRAN
DECLARE @x as int;
SET @x = 0;

WHILE (@x < 100000)
BEGIN
	INSERT INTO howboutthemcowboys VALUES (@x, 'Jason Witten returns in 2019')
	SET @x = @x + 1
END

COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:30.419

Step 2: Delete all the rows in the table in an uncommitted transaction. Took 41 secs, double the time to create it??

In [26]:
use master;
go

alter DATABASE gocowboys set accelerated_database_recovery = OFF
go

use gocowboys;
GO

BEGIN TRAN
DELETE from howboutthemcowboys;
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:03.444

Step 3: Check how much transaction log space is used. 4GB used out of total of 5GB. The source notebook published under MSFT account shows that it used 91% but in my case it only 78% has been used. Since MSFT e.g. code is running on windows and mine in a linux docker container, does it mean log file space is more efficiently used in Linux filesystem??

In [27]:
SELECT used_log_space_in_bytes/(1000*1000*1000) as log_used_GB
,total_log_size_in_bytes/(1000*1000*1000) as log_totalSize_GB
,* FROM sys.dm_db_log_space_usage;
GO


(1 row affected)

Total execution time: 00:00:00.057

log_used_GB,log_totalSize_GB,database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
4,5,18,5368700928,4917342208,91.59277,1865355264


**Step 4: Does a checkpoint truncate the transaction log?**

Normally for a database with simple recovery a CHECKPOINT would truncate the log. In case I am running SQL Server on linux (container), it makes no difference but if i run the same code on Windows, the used percentage drops from 91% to 79%. Why??

In [28]:
CHECKPOINT;
GO

SELECT used_log_space_in_bytes/(1000*1000*1000) as log_used_GB
,total_log_size_in_bytes/(1000*1000*1000) as log_totalSize_GB
,* FROM sys.dm_db_log_space_usage;
GO


Commands completed successfully.

(1 row affected)

Total execution time: 00:00:02.524

log_used_GB,log_totalSize_GB,database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
4,5,18,5368700928,4246458368,79.09657,1015545856


Step 5: How long does it take to rollback all the deletes?

In [29]:
ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:13.499

**Step 6: What is the log space usage after a CHECKPOINT?**

Because the active transaciton is rolled back, a CHECKPOINT will allow the log to be truncated.

In [30]:
CHECKPOINT;
GO

SELECT used_log_space_in_bytes/(1000*1000*1000) as log_used_GB
,total_log_size_in_bytes/(1000*1000*1000) as log_totalSize_GB
,* FROM sys.dm_db_log_space_usage;
GO


Commands completed successfully.

(1 row affected)

Total execution time: 00:00:02.235

log_used_GB,log_totalSize_GB,database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
0,5,18,5368700928,88764416,1.653369,110592


**Step 7: Turn on Accelerated Database Recovery**


In [31]:
USE master
GO

ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.036

**Step 8: Try to delete all the rows again under a transaction**

In [32]:
USE gocowboys;
GO

BEGIN TRAN
    DELETE from howboutthemcowboys;
GO

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:02.920

**Step 9: Check log space usage before and after a CHECKPOINT**

In [33]:
SELECT used_log_space_in_bytes/(1000*1000*1000) as log_used_GB
,total_log_size_in_bytes/(1000*1000*1000) as log_totalSize_GB
,* FROM sys.dm_db_log_space_usage;
GO

CHECKPOINT;
GO

SELECT used_log_space_in_bytes/(1000*1000*1000) as log_used_GB
,total_log_size_in_bytes/(1000*1000*1000) as log_totalSize_GB
,* FROM sys.dm_db_log_space_usage;
GO

(1 row affected)

log_used_GB,log_totalSize_GB,database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
1,5,18,5368700928,1079373824,20.10493,768110592


Commands completed successfully.

(1 row affected)

Total execution time: 00:00:02.646

log_used_GB,log_totalSize_GB,database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
0,5,18,5368700928,408522752,7.609341,102400


**Step 10: How fast is a rollback?**

In [34]:
ROLLBACK TRAN;
GO

Commands completed successfully.

Total execution time: 00:00:00.001